In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# LDDMM landmark dynamics

In [ ]:
%cd ..
from src.manifolds.landmarks import *
M = landmarks(21)
print(M)

from src.plotting import *

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

In [ ]:
# example configuration
M.k_sigma.set_value(np.diag((.5,.5)))

q = M.coordsf(np.vstack((
    np.stack((-.5+np.zeros(int((M.N.eval()-3)/2)),np.linspace(-1.,1.,int((M.N.eval()-3)/2))),axis=1),
    np.array([[-.25,0.]]),
    np.array([[0.,0.]]),
    np.array([[.25,0.]]),
    np.stack((.5+np.zeros(int((M.N.eval()-3)/2)),np.linspace(-1.,1.,int((M.N.eval()-3)/2))),axis=1),    
)).flatten())
v = .5*np.vstack((np.ones(M.N.eval()),np.ones(M.N.eval()))).T.flatten()
p = M.flatf(q,v.flatten())
print("q = ", q)
print("p = ", p)
M.plot()
M.plotx(q)
plt.axis('off')
plt.savefig('shape.pdf')

## Brownian Motion

In [ ]:
# coordinate form
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

(ts,qs,charts) = M.Brownian_coordsf(q,.2*dWsf(M.dim.eval()))
M.plot()
M.plot_path(zip(qs,charts))
plt.axis('off')
plt.savefig('Brownian.pdf')
plt.show()

# Langevin equations
see https://arxiv.org/abs/1605.09276

In [ ]:
from src.stochastics import Langevin
Langevin.initialize(M)

dWs = dWsf(M.dim.eval())

(ts,qps,charts) = M.Langevinf(q,p,.5,.25,dWs)
M.plot()
M.plot_path(zip(qps[:,0,:],charts))
plt.axis('off')
plt.savefig('Langevin.pdf')
plt.show()

(ts,qps,charts) = M.Langevinf(q,p,0.,.25,dWs)
M.plot()
M.plot_path(zip(qps[:,0,:],charts))
plt.axis('off')
plt.savefig('Lagrangian.pdf')
plt.show()

## Stochastic EPDiff / Eulerian

In [ ]:
# define noise field grid
minx = -1; maxx = 1
miny = -1; maxy = 1
pts = 3

X, Y = np.meshgrid(np.linspace(minx,maxx,pts),np.linspace(miny,maxy,pts))
xy = np.vstack([X.ravel(), Y.ravel()]).T
sigmas_x = np.hstack((xy,xy)).reshape((-1,2))
sigmas_a = .05*np.tile(np.eye(2),(sigmas_x.shape[0]//2,1))
#sigmas_x = np.array([[0.,0.]])
#sigmas_a = np.array([[.1,0.]])
J = sigmas_x.shape[0]

# noise kernels
k_alpha = 1.
k_sigma = np.diag(((maxx-minx)/(pts-1),(maxy-miny)/(pts-1)))
inv_k_sigma = theano.tensor.nlinalg.MatrixInverse()(k_sigma)
k = lambda x: k_alpha*T.exp(-.5*T.sqr(T.tensordot(x,inv_k_sigma,(0 if x.type == T.vector().type else 2,1))).sum(0 if x.type == T.vector().type else 2))

# initialize with specified noise kernel
from src.stochastics import Eulerian
Eulerian.initialize(M,k=k)

# integrate
(ts,qps,charts) = M.Eulerianf(q,p,sigmas_x.flatten(),sigmas_a.flatten(),dWsf(J))
M.plot()
M.plot_path(zip(qps[:,0,:],charts))
plt.axis('off')
plt.plot(sigmas_x[:,0],sigmas_x[:,1],'x')
plt.savefig('Eulerian.pdf')
plt.show()

plt.figure(figsize=(4,4))
M.plot()
M.plot_path(zip(qps[:,0,:],charts), color='k', linewidth=0.5)
plt.plot(sigmas_x[:,0],sigmas_x[:,1],'x')
plt.axis('off')
plt.axis([-1.2, 1.2, -1.2, 1.2])
plt.savefig('Eulerian.svg')
plt.show()